In [ ]:
import os
import qubic
import healpy as hp
import numpy as np
import pylab as plt
import matplotlib as mpl
from matplotlib.pyplot import *
from matplotlib import rc
#from qubicpack import qubicpack as qp
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

import sys
sys.path.append('/home/martin/QUBIC/qubiclouise/qubic/scripts/Calibration')
sys.path.append('/home/martin/QUBIC/qubiccalibration/qubicpack')

from qubic import myinstrument as myinst
from qubic.myinstrument import QubicInstrument

#import fibtools as ft
#import plotters as p
#import lin_lib as ll
#import demodulation_lib as dl
#import satorchipy as stpy
from pysimulators import FitsArray

import glob
import matplotlib.mlab as mlab
import scipy.ndimage.filters as f
import string
import scipy.signal as scsig
from scipy import interpolate
import datetime as dt
%matplotlib inline

def select_det(q,id):
    # Select detector number id from a QubicInstrument instance q
    # Return:
    #     A QubicInstrument object with only detector id (insted full detector array)
    id=[id]
    detector_i = q.detector[id]
    q.detector = detector_i
    return(q)


# +
#mpl.style.use('classic')
name='test_scan_source'
resultDir='%s'%name


In [ ]:
#alaImager:
#    - True: If you want the results for d['kind'] = 'I' and only one detector.
alaImager=False
component=0
sel_det=False #If you want to select one detector
oneComponent=True # = True if you want to work only for I component
# -

try:
    os.makedirs(resultDir)
except:
    pass


# INSTRUMENT
d = qubic.qubicdict.qubicDict()
d.read_from_file('global_source_allDet.dict')

q = qubic.QubicInstrument(d)
#plt.title('Detector array')
#plt.plot(q.detector.center[did_det['93']][0],q.detector.center[did_det['93']][1], 'ko')
#q.detector.plot()

if sel_det:
    TESid = 93
    #id_det=180 # Detector number to use, 180 == 93 from real detector?
    did_det={'93':180, '94':197, '95':214, '96':231}#180sim == 93real, 197sim == 94 real, 214sim == 95real, 231sim == 96real 
    q=select_det(q,did_det['{}'.format(TESid)])

In [ ]:
pp= qubic.get_pointing(d)

In [ ]:
s = qubic.QubicScene(d)

sb = q.get_synthbeam(s, 290, detpos=q.detector.center[0])
xr=0.1*np.max(sb)
sb_img=hp.gnomview(sb, rot=[0,90], xsize=500, reso=5, min=-xr, max=xr,title='Input ', 
                   return_projected_map=True,hold=False,cmap='viridis')
#plt.show()
plt.clf()
plt.imshow(sb_img, cmap='viridis')
plt.colorbar()
plt.show()


In [ ]:
fix_azimuth=d['fix_azimuth']
m0=np.zeros(12*d['nside']**2)
x0=np.zeros((len(m0),3))
id=hp.pixelfunc.ang2pix(d['nside'], fix_azimuth['az'], fix_azimuth['el'],lonlat=True)
print(id)
source=m0*0
source[id]=1

freq = 150 # GHz
conv = 61.3474
#source = hp.sphtfunc.smoothing(source,fwhm=np.deg2rad(conv/freq))#
source = hp.sphtfunc.smoothing(source,fwhm=np.deg2rad(1.07))# 1.07 == mean of the fwhm fitted in fitted_peaks.fits
x0[:,component]=source

if pp.fix_az:
    center = (fix_azimuth['az'],fix_azimuth['el'])
else:
    center = qubic.equ2gal(d['RA_center'], d['DEC_center'])

hp.gnomview(x0[:,0], rot=center,reso = 5,cmap='viridis',xsize=400)
plt.show()


In [ ]:
# Data already procceced and binned. The binned is 200
#TESid = TESid
#filename = []
dirflat = '/home/martin/QUBIC/qubiccalibration/Data/Calib-TD/150GHz-2019-04-06/Flat/'
filename = []
#save current directory
currdir = os.getcwd()
#Change working directory and load filenames
os.chdir(dirflat)
datafiles = glob.glob('*.fits')
for ifile in datafiles: 
    filename.append(dirflat+ifile)             #'imgflat_TESNum_{}.fits'.format(TESid)
datamap = []
for ifile in filename:
    datamap.append(FitsArray(ifile))
# go back previous directory
os.chdir(currdir)
# Same pointing for each detector
az = FitsArray(dirflat+'azimuth.fits')
el = FitsArray(dirflat+'elevation.fits')

#read xcenter, ycenter, amplitude and fwhm of the desired detector:
peaks = np.array(FitsArray('/home/martin/QUBIC/qubiclouise/qubic/scripts/pointsource/fitted_peaks.fits'))
peaks[:,0,:] = peaks[:,0,:]/np.cos(np.radians(50))

In [ ]:
tesid = 92
plot(peaks[tesid,0,:], peaks[tesid,1,:],'ko')

In [ ]:
"""Do again TOD simulation with the new pointing strategy.
Use the new pointing for TOD data too.."""

In [ ]:
def hall_pointing(az, el, angspeed_psi, maxpsi,
                 date_obs=None, latitude=None, longitude=None,fix_azimuth=None,random_hwp=True):
    #This method will reproduce the pointing that is used in the hall to take the data. Will start from bottom
    #left and will go up at fixed elevation.
    
    #nsamples = 2*len(az)*len(el) # commented bucause not go and back
    nsamples = len(az)*len(el)
    print('nsamples = ', nsamples)
    pp = qubic.QubicSampling(nsamples,date_obs=date_obs, period=0.1, latitude=latitude,longitude=longitude)
    
    #Comented because we do not go and back in simulations.. 
    #mult_el = []
    #for eachEl in el:
    #    mult_el.append(np.tile(eachEl, 2*len(az)))
    # Azimuth go and back and same elevation. 
    #az_back = az[::-1]
    #az = list(az)
    #az.extend(az_back)
    #mult_az = np.tile(az, len(el))
    #print(i,np.asarray(mult_el).ravel().shape)
    #pp.elevation = np.asarray(mult_el).ravel()
    #pp.azimuth = np.asarray(mult_az).ravel()
    
    mult_el = []
    for eachEl in el:
        mult_el.extend(np.tile(eachEl, len(az)))
    mult_az = []
    mult_az.append(np.tile(az, len(el)))
    #print(len(mult_el))
    pp.elevation = np.asarray(mult_el)#az2d.ravel()
    pp.azimuth = np.asarray(mult_az[0])#el2d.ravel()
    
    ### scan psi as well,
    pitch = pp.time * angspeed_psi
    pitch = pitch % (4 * maxpsi)
    mask = pitch > (2 * maxpsi)
    pitch[mask] = -pitch[mask] + 4 * maxpsi
    pitch -= maxpsi
    
    pp.pitch = pitch
    
    if random_hwp:
        pp.angle_hwp = np.random.random_integers(0, 7, nsamples) * 11.25
        
    if fix_azimuth['apply']:
        pp.fix_az=True
        if fix_azimuth['fix_hwp']:
            pp.angle_hwp=pp.pitch*0+ 11.25
        if fix_azimuth['fix_pitch']:
            pp.pitch= 0
    else:
        pp.fix_az=False

    return pp
    #print(pp.elevation)#, len(pp.elevation))
    

#p = hall_pointing(az[::-1], el[::-1], d['angspeed_psi'], d['maxpsi'], fix_azimuth=d['fix_azimuth'])
p = hall_pointing(az, el, d['angspeed_psi'], d['maxpsi'], fix_azimuth=d['fix_azimuth'])

#print aztest

In [ ]:
type(datamap[0])

In [ ]:
az2d,el2d = np.meshgrid(az,el)
myTOD = np.zeros((len(datamap),len(datamap[0].ravel())))
print(np.shape(myTOD), len(datamap[0].ravel()))
for j,idata in enumerate(datamap):
    myTOD[j,:] = np.ravel(idata)[0]

In [ ]:
figure(figsize=(8,6))
plt.imshow(datamap[200], cmap='viridis')
plt.show()


In [ ]:
#Simulo un nuevo TOD con el pointing de la observacion 
#reload(qubic)
sima = qubic.QubicAcquisition(q, p, s, d)
simTOD, maps_conv_p = sima.get_observation(x0, convolution = True, noiseless=True)

In [ ]:
simTOD.shape

In [ ]:
datamap = np.array(datamap)

displ = False

if displ:
    ## REVISAR ALGORITMOK
    idx = 0
    newmap = np.reshape(myTOD[idx], datamap[idx].shape)
    #invierto el TOD simulado
    for j,_ in enumerate(simTOD):
        simTOD[j,:] = simTOD[j,::-1]
    simmap = np.reshape(simTOD[idx], datamap[idx].shape)
    figure(figsize=(12,12))
    subplot(221)
    imshow(newmap, extent=[az.min(),az.max(), el.min(),el.max()],cmap='viridis')
    subplot(222)
    imshow(simmap*1e27, extent=[az.min(),az.max(), el.min(),el.max()], cmap='viridis')
    subplot(223)
    plot(myTOD)
    subplot(224)
    plot(simTOD[0]*1e27)
    show()

In [ ]:
# I have 10 detectors more than in simulated TOD...
print(simTOD.shape, myTOD.shape)

In [ ]:
"""To find the locations of the peaks in the TOD space"""

az2dravel = az2d.ravel()[::-1]
el2dravel = el2d.ravel()[::-1]

locpeak_tod = []
azloc = np.zeros()
print('====')
for ipeak in peaks:
    for i in xrange(0,len(ipeak[0])):
        azloc = np.where(abs(az2dravel-ipeak[0,i]) < 0.12 )
        elloc = np.where(abs(el2dravel-ipeak[1,i]) < 0.12 )
        az1 = az2dravel[azloc]
        el1 = el2dravel[elloc]
    
        for each in azloc[0]:
            if each in elloc[0]:
                locpeak_tod.append(each)
        
print('====')

print(len(locpeak_tod), locpeak_tod)
peaktod = np.zeros((len(myTOD,)))
peaktod[locpeak_tod] = np.max(myTOD)

print(az2d.ravel()[locpeak_tod])
print(el2d.ravel()[locpeak_tod])

In [ ]:

thetas = np.array([[0.03281347, 0.11251228, 0.14091657 , 0.18485994, 0.17793436,
0.14746803, 0.12888845, 0.18494313, 0.242361  , 0.25671088,
0.26956301, 0.28452484, 0.2904603 ]])
thetas_def = np.array([[ 0.01131323,  0.13182682,  0.14369991,  0.14369991,  0.15468746,
         0.19529527,  0.19529527,  0.21161855,  0.21161855,  0.27776151,
         0.28977923,  0.28977923,  0.30137091,  0.31428866]])
phis =np.array([[ 2.83409643, -0.97533545,  1.06666189,  0.04872912, -2.5518964 ,
 2.47655669, -2.06418194,  1.775554977581,  2.90827761, -0.86898177,
 0.90775038, -0.33751121,  0.3902112 ]])
phis_def = np.array([[ 2.35619449, -0.78539816,  0.86447862, -2.43527494,  2.35619449,
        -1.61203054,  0.04123422,  3.10349837,  1.60889061, -0.78539816,
        -2.39579663,  0.8250003 ,  2.35619449, -1.26541229]])
vals = np.array([[32.14002372, 33.95124927, 32.61291557,23.3299855 , 21.66893532,
 7.77258269,  9.33062719,  30.42635041,  3.25307711,  1.53202714,
 1.06429809,  0.68101577,  0.56667237]])
vals_def = np.array([[ 52.9271193 ,  20.90234505,  17.52582023,  17.52582023,
         14.68904366,   6.83243483,   6.83243483,   4.7776674 ,
          4.7776674 ,   0.83580943,   0.57886246,   0.57886246,
          0.40021914,   0.26078313]])
 

In [ ]:
# Conventions:
#    theta's angles: 90 - elevation
#    phi's angles: azimuth angle
# So:
#    x = sin(theta)*cos(phi)
#    y = sin(theta)*sin(phi)
#    z = cos(theta)
#

def thph2uv_one(th,ph):
    sth = np.sin(th)
    cth = np.cos(th)
    sph = np.sin(ph)
    cph = np.cos(ph)
    return np.array([sth*cph, sth*sph,cth])

def uv2thph_one(uv):
    r = np.sum(uv**2)
    th = np.nan_to_num(np.arccos(uv[2]/r))
    ph = np.arctan2(uv[1],uv[0])
    return np.array([th,ph])

def rotmatX(th):
    cth = np.cos(th)
    sth = np.sin(th)
    rotmat = np.array([[1, 0, 0],[0, cth, -sth],[0,sth,cth]])
    return rotmat

def rotmatY(th):
    cth = np.cos(th)
    sth = np.sin(th)
    rotmat = np.array([[cth, 0, sth],[0, 1, 0],[-sth,0,cth]])
    return rotmat

def rotmatZ(th):
    cth = np.cos(th)
    sth = np.sin(th)
    rotmat = np.array([[cth, -sth, 0], [sth, cth, 0], [0, 0, 1]])
    return rotmat

def neworder(arr,loc):
    #arr has the order of JCh fit
    # loc has de convertion betwen JCh order and myinstrument order. I need myinstrument order to put it there
    auxarr = np.zeros(arr.shape)
    for i, each in enumerate(loc):
        auxarr[i] = arr[each]
    return auxarr

In [ ]:
thph2uv_one(0.03,2.93)
print('az peak', np.radians(peak95[0,:]))
print('el peak', np.radians(peak95[1,:]))

In [ ]:
print '======= Rotation from fit points to theoretical reference system'
print('Position of the central peak (fifth in fit array), (az,el) = {}, {}'.format(peak95[0,4],peak95[1,4]))

elcenter = el[len(el)/2]#50.#+(offele-50.)#peak95[1,4]#
azcenter = az[len(az)/2]#0.#-(offaz-0)peak95[0,4]#

print('Central position in the observed region, (az,el) = {},{}'.format(azcenter,elcenter))

elcenter = el[len(el)/2+3]
azcenter = az[len(el)/2-2]

print('Closest position between center of the observed region and (0.,50.) = {}, {}'.format(azcenter, elcenter))
az0 = peak95[0,:]#*np.cos(np.radians(peak95[1,:]))
el0 = peak95[1,:]

print('======Try angle betwen red and green (Martin notes)')

xaxisred = thph2uv_one(np.radians(90-50), np.radians(0))
xaxisgreen = thph2uv_one(np.radians(90-50.81),np.radians(0.12))
#print(xaxisred)
#print(xaxisgreen)
angulo = np.arccos(np.dot(xaxisred,xaxisgreen)) #angular distance between xaxisred and xaxisgreen (offset?)
import math as m
angulo2 = m.atan2(np.radians(0.81),np.radians(0.12)*np.cos(np.radians(50.0)) ) # approximation
angulo2 = 90-np.degrees(angulo2)
print(angulo2)
print('======Try angle betwen red and green (Martin notes)')

th0, ph0 = np.pi/2-np.radians(el0), np.radians(az0)
print('phis,theta_0, ', ph0[0], th0[0])
pos0 = []
for j in xrange(0,len(el0)):
    pos0.append(thph2uv_one(th0[j],ph0[j]))

posF3 =[]

rott = 1

if rott:
    for j in xrange(0,len(el0)):
        # rotation from center of the field in az,el to 
        posFx = np.dot(rotmatZ(-(np.radians(azcenter)*np.cos(np.radians(elcenter)) )), np.asarray(pos0[j]))
        posFxx = np.dot(rotmatY(-(np.radians(90-elcenter))), posFx)
        posFz= np.dot(rotmatX(np.radians(angulo2) )  , np.asarray(posFxx)  )
        posF0 = np.dot(rotmatZ(angulo ) , np.asarray(posFz)  ) 
        posF3.append(np.dot(rotmatX(-np.radians(angulo2) )  , np.asarray(posF0)  ))

else:
    for j in xrange(0,len(el0)):
        posF0 = np.dot(rotmatZ(-(np.radians(azcenter)*np.cos(np.radians(elcenter)) )), np.asarray(pos0[j]))
        posF3.append(np.dot(rotmatY(-(np.radians(90-elcenter))), posF0))

newcoord = []
for i in xrange(0,len(posF3)):
    newcoord.append(uv2thph_one(posF3[i]))


newth = np.array(newcoord).T[0]#+newthoff
newphi = +np.array(newcoord).T[1]*np.cos(newth)#+newphoff

#np.savetxt('leb_defsynthbeam.txt',[th0.T, ph0.T])
np.savetxt('leb2.txt', [newth,newphi])

clf()
figure(figsize=(12,12))

subplot(122, projection = 'polar')
for i in xrange(0,len(newcoord)):
    plot(ph0[i],th0[i], 'ro')
    #plot(newcoord[i][1],newcoord[i][0],'ko')
    plot(newphi,newth,'ko')
    plot(0,np.pi/2,'go')

print '======='

idp=[4,7,5,8,3,1,6,2,0]

#print('ppp',newth)
#print(neworder(newth,idp))
#print(phis[0])
#neworder(newth,idp)

In [ ]:
# inverse rotation: from theoretical to FOV
uv_def = thph2uv_one(thetas_def,phis_def).T
#Take the fisrt 9 peaks
newarr = np.zeros((9,3))
for i,each in enumerate(uv_def[:9]):
    newarr[i] = each[0]

posF04 = []
# take the first 9 elements of the default values
thetas9 = thetas_def[0][:9]
phis9 = phis_def[0][:9]

for j,each in enumerate(newarr):
    # rotation from center of the field in az,el to 
    posF00 = np.dot(rotmatZ(-(np.pi-phis9[0]) )  , each  )
    posF0i = np.dot(rotmatX(+thetas9[0] ) , np.asarray(posF00)  )
    #posF01 = np.dot(rotmatX(-(np.pi/2 - np.radians(40)) ) , np.asarray(posF0i)  )
    posF04.append(np.dot(rotmatX(-( np.radians(40))/np.cos(np.radians(50)) ) , np.asarray(posF0i)  ) )
    #posF01i = np.dot(rotmatX(-thetas9[0] ) , np.asarray(posF01)  )
    
    #posF02 = np.dot(rotmatX(-np.radians(angulo2) )  , np.asarray(posF01  ) )
    #posF03 = np.dot(rotmatY(-(np.radians(90-elcenter))), posF02  )
    #posF04.append(np.dot(rotmatZ(+(np.radians(azcenter)*np.cos(np.radians(elcenter)) )), np.asarray(posF03)) )

nthph = uv2thph_one(np.asarray(posF04).T)
#print nthph[0]
np.savetxt('leb_defsynthbeam.txt',[thetas9, phis9])
np.savetxt('leb-inverse.txt', [nthph[0],nthph[1]])

clf()
figure(figsize=(12,12))

subplot(122, projection = 'polar')

for i in xrange(0,9):
    plot(phis_def[0][i],thetas_def[0][i], 'ro')
    plot(nthph[1][i],nthph[0][i],'ko')
    #plot(0,np.pi/2,'go')
    #legend()

sin = np.sin
cos = np.cos
pi = np.pi
r=1.
phim, thetam = np.mgrid[0.0:pi:100j, 0.0:2.0*pi:100j]
xi = r*sin(phim)*cos(thetam)
yi = r*sin(phim)*sin(thetam)
zi = r*cos(phim)

data = np.genfromtxt('leb_defsynthbeam.txt', unpack=True)
newdata2 = np.genfromtxt('leb-inverse.txt', unpack = True)

thetai, phii = np.hsplit(data, 2) 
newthetai, newphii = np.hsplit(newdata2,2)

xxi = sin(thetai)*cos(phii)
yyi = sin(thetai)*sin(phii)
zzi = cos(thetai)
xxxi = sin(newthetai)*cos(newphii)
yyyi = sin(newthetai)*sin(newphii)
zzzi = cos(newthetai)

el2=50.
cel=np.radians(90-el2)
az2 = 0.
px = sin(cel)*cos(az2)
py = sin(cel)*sin(az2)
pz = cos(cel)
#Set colours and render
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(xi, yi, zi,  rstride=1, cstride=1, color='c', alpha=0.3, linewidth=0)
ax.scatter(xxi,yyi,zzi,color="r",s=20)
ax.scatter(xxxi,yyyi,zzzi,color="k", s =20)
ax.scatter(px,py,pz,color="k", s = 30)

ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.set_zlim([-1,1])

ax.set_aspect("equal")
plt.tight_layout()
plt.show()

In [ ]:
# Try to chenge the peaks location inside /usr/local/lib/python2.7/dist-packages/qubic/myinstrument.py

reload(myinst)
myq = myinst.QubicInstrument(d)

if sel_det:
    myq=select_det(myq,did_det['{}'.format(TESid)])

def bla(myq,p):
    horn = getattr(myq, 'horn', None)
    primary_beam = getattr(myq, 'primary_beam', None)
    if p.fix_az:
        rotation = p.cartesian_horizontal2instrument
    else:
        rotation = p.cartesian_galactic2instrument
    
    qubinst, thetas,phis,vals = QubicInstrument._get_projection_operator(
            rotation, s, myq.filter.nu, myq.detector.center,
            myq.synthbeam, horn, primary_beam, verbose=True)
    return thetas, phis, vals

#thetas_def, phis_def, vals_def = bla(myq,p)

In [ ]:
reload(myinst)

myq = myinst.QubicInstrument(d)

if sel_det:
    myq=select_det(myq,did_det['{}'.format(TESid)])

mya = qubic.QubicAcquisition(myq, p, s, d)
mytod, my_conv = mya.get_observation(x0, convolution = True, noiseless = False)
#mytod[0] = mytod[0,::-1]
figure(figsize=(14,5))
plot(myTOD, 'b-')
#roll tod
#mytod = np.roll(mytod, len(mytod)+4*len(az))
mytod[0] = 3.3*mytod[0]
#plot(mytod[0], 'g-')

plot(1e-12*peaktod[:], 'r--')


In [ ]:
# Create mytod inverted
#mytodinv = mytod[::-1]
#fa = np.cos(np.radians(50))

milimx,malimx,milimy,malimy = -26,26,36,65
setlim=1
clf()
figure(figsize=(18,16))
linen = 200
pixi=el.shape[0]*linen

subplot(221)
if setlim:
    xlim(milimx,malimx)
    ylim(milimy,malimy)
title('TOD for PS, instrument myq (TES={})'.format(TESid))
newnew = mytod[0][::-1]
imshow(np.reshape(mytod[0],(el.shape[0],linen)), extent=[az.min(),#*np.cos(np.radians(88)),
                                                           az.max(),#*np.cos(np.radians(50)), 
                                                           el.min(),el.max()],cmap = 'viridis')

for i in xrange(0,9):#len(locpeak_tod)):
    plot(peak95[0,i],peak95[1,i],'wo')
    text(peak95[0,i]*0.96, peak95[1,i]*0.95,'{}'.format(i))
plot(0,50,'ro')
    
subplot(222)
if setlim:
    xlim(milimx,malimx)
    ylim(milimy,malimy)
title('real TOD (TES={})'.format(TESid))
myim2 = imshow(np.reshape(myTOD[:],(el.shape[0],linen)),extent=[az.min(),#*np.cos(np.radians(50)),
                                                      az.max(),#*np.cos(np.radians(50)),
                                                      el.min(),el.max()],cmap = 'viridis')
for i in xrange(0,9):#len(locpeak_tod)):
    plot(peak95[0,i],peak95[1,i],'wo')
    text(peak95[0,i]*0.96, peak95[1,i]*0.95,'{}'.format(i))
    
subplot(223)
if setlim:
    xlim(milimx,malimx)
    ylim(milimy,malimy)
title('TOD for PS instrument q (TES={})'.format(TESid))
imshow(np.reshape(simTOD[0][:],(el.shape[0],linen)), extent=[az.min(),#*np.cos(np.radians(50)),
                                                            az.max(),#*np.cos(np.radians(50))
                                                            el.min(),el.max()],cmap = 'viridis')
plot(0,50,'ro')

for i in xrange(0,9):
    plot(peak95[0,i],peak95[1,i],'wo')
    text(peak95[0,i]*0.96, peak95[1,i]*0.95,'{}'.format(peak95[1,i]))#plot(peak95[0,:],peak95[1,:],'wo')
#savefig('TES={}'.format(TESid))

In [ ]:
#myarec = qubic.QubicAcquisition(myq, p, s,d)
#values_tod = np.reshape(np.asarray(values),(1,len(values)))
d['tol'] = 1e-5
maps_recon_real = mya.tod2map([myTOD,], d, cov=None)

""" using sima = qubic.QubicAcquisition(q, p, s, d)"""
maps_recon_simTOD = sima.tod2map(simTOD, d, cov=None)
#mas_recon_simu = sima.tod2map(mytod, d, cov=None)
maps_recon_mytod = sima.tod2map(mytod, d, cov=None)

In [ ]:
figure(figsize=(15,10))
resize=400
#Read Healpy projection
tes93healpy = qubic.io.read_map(dirflat+'../Healpix/healpix_TESNum_{}.fits'.format(TESid))

map1=hp.gnomview(maps_recon_real[:,0],sub=(1,3,1),title='Recons (real TOD, modify myinst)',#min=-0.1,max=0.5,
             rot=center,reso = 5, xsize = resize, return_projected_map=True)
map2=hp.gnomview(x0[:,0], sub=(1,3,2),rot=center, reso = 5, xsize=resize, return_projected_map=True)
#hp.gnomview(maps_recon_simTOD[:,0],sub=(1,3,2),title='Recons with sim TOD',
#            rot=center,reso = 5, xsize = resize)
hp.gnomview(tes93healpy,sub=(1,3,3),title='Healpy proj TES = {}'.format(TESid), reso = 5, xsize = resize)
#hp.gnomview( (mas_recon_simu[:,0]-(maps_recon_real[:,0])),#min=-0.1,max=0.1,
#           sub=(2,2,4),title='Diff.',rot=center,reso = 5, xsize = 400)


In [ ]:
az[len(az)/2], el[len(el)/2]+1, peak95[1,4], peak95[0,4] 

Why will not work the rotation method? 

The rotation fo the reference system will move that absolute system where the position of the points are defined. But we do not have the same relative position in between the points and it will no be able to do a rotation

In [ ]:
ttt,ppp = np.loadtxt('leb2.txt')
print ttt, np.radians(50)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

sin = np.sin
cos = np.cos
pi = np.pi
r=1.
phim, thetam = np.mgrid[0.0:pi:100j, 0.0:2.0*pi:100j]
xi = r*sin(phim)*cos(thetam)
yi = r*sin(phim)*sin(thetam)
zi = r*cos(phim)

data = np.genfromtxt('leb.txt', unpack=True)
newdata = np.genfromtxt('leb2.txt', unpack = True)
thetai, phii = np.hsplit(data, 2) 
newthetai, newphii = np.hsplit(newdata,2)

#theta = theta * pi / 180.0
#phi = phi * pi / 180.0
xxi = sin(thetai)*cos(phii)
yyi = sin(thetai)*sin(phii)
zzi = cos(thetai)
xxxi = sin(newthetai)*cos(newphii)
yyyi = sin(newthetai)*sin(newphii)
zzzi = cos(newthetai)

el2=50.
cel=np.radians(90-el2)
az2 = 0.
px = sin(cel)*cos(az2)
py = sin(cel)*sin(az2)
pz = cos(cel)
#Set colours and render
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(xi, yi, zi,  rstride=1, cstride=1, color='c', alpha=0.3, linewidth=0)
ax.scatter(0,0,1, color="r", s=20)
ax.scatter(xxi,yyi,zzi,color="k",s=20)
ax.scatter(xxxi,yyyi,zzzi,color="g", s =20)
ax.scatter(px,py,pz,color="k", s = 30)
#ax.scatter(uv_def, color = "w",s =30)
#ax.quiver(0,0,0,px,py,pz)
ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.set_zlim([-1,1])
ax.set_aspect("equal")
plt.tight_layout()
plt.show()

In [ ]:
elcenter = np.radians(50.)
azcenter = np.radians(0.)
cel = np.cos(elcenter)
caz = np.cos(azcenter)
sel = np.sin(elcenter)
saz = np.sin(azcenter)
xcenter = np.array([cel*caz,cel*saz,sel])
idp=[4,7,5,8,3,1,6,2,0]
angulo = []
for iidp in idp:
    cenpeakel = np.radians(peak95[1,iidp])
    cenpeakaz = np.radians(peak95[0,iidp])

    celp = np.cos(cenpeakel)
    cazp = np.cos(cenpeakaz)
    selp = np.sin(cenpeakel)
    sazp = np.sin(cenpeakaz)
    xpeak = np.array([celp*cazp,celp*sazp,selp])
    print np.arccos(np.dot(xcenter,xpeak))
    angulo.append(np.arccos(np.dot(xcenter,xpeak)))
print(angulo)

In [ ]:
# Tengo las posiciones. 

In [ ]:
map1=map1/np.max(map1)
map2=map2/np.max(map2)
figure(figsize=(14,8))
subplot(131)
imshow(abs(map1))
colorbar()
subplot(132)
imshow(abs(map2))
colorbar()
subplot(133)
colorbar()
imshow(abs(map2)-abs(map1))

In [ ]:
print(np.shape(maps_recon_new),np.shape(maps_recon))
cmapcode = 'viridis'
resol = 5
size = 400
plt.figure(figsize=(10,10))
hp.gnomview(x0[:,0], sub = (2,2,1), rot=center, reso = resol, title = 'Input map', cmap=cmapcode, xsize = size)

hp.gnomview(maps_recon_new[:,0], sub = (2,2,2), rot= center, reso = resol, xsize = size, 
            title = 'Rec from real TOD', cmap=cmapcode)

hp.gnomview(maps_recon[:,0], rot=center, reso=resol, sub = (2,2,3), title='Recons from simulation ', 
            cmap=cmapcode, xsize = size)

hp.gnomview(maps_recon[:,0]- maps_recon_new[:,0], sub=(2,2,4), rot = center, xsize = size,
            reso = resol, title = 'Diff theoretical TOD- real TOD', cmap = cmapcode)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.subplot(121)
plt.plot(p.azimuth[0:1200])
plt.subplot(122)
plt.plot(p.elevation[0:1200])

plt.show()

In [ ]:
# Test of sampling ...
step = 100
for i in range(2):
    plt.clf()
    plt.xlim(-27,27)
    plt.ylim(33,38)
    plt.plot(p.azimuth[0:step*(i+1)],p.elevation[0:step*(i+1)],'bo')
    plt.pause(1.)
    plt.show()
for i in range(2):
    plt.clf()
    plt.xlim(-27,27)
    plt.ylim(33,38)
    plt.plot(p.azimuth[0:200],p.elevation[0:200],'bo')
    plt.plot(p.azimuth[step*(2):step*(i+3)],0.03+p.elevation[step*(2):step*(i+3)],'ro')
    plt.pause(1.)
    plt.show()

In [ ]:
# I belive that I already have the pointing strategy done. We start to think how recons the map... 
# First, I have the TOD
#
#
print(np.shape(datamap))
print(2*np.shape(datamap.ravel())[0])
print(type(datamap))



In [ ]:
#invierto cada linea de elevacion constante
keep_rev_az = []
for azi in datamap:
    keep_rev_az.append(azi[::-1])

#intercalo la data de lectura a elevacion constante para simular el ida y vuelta a elevacion constante
newTOD = np.empty((150,400))
for i,each in enumerate(datamap):
    auxlist = []
    auxlist += list(each)
    auxlist += keep_rev_az[i]
    newTOD[i,:200] = each
    newTOD[i,200:] = keep_rev_az[i]
    
datamap_copy = np.copy(datamap)
#plt.plot(datamap.ravel())
#plt.plot(datamap_copy[::-1].ravel()-2e4,'r-')
#plt.plot(datamap_copy.ravel()[::-1]-4e4,'g-')
#plt.show()

In [ ]:
print(np.shape(newTOD))
#veo si cada elemento del nuevo TOD está espejado respecto a 200
plt.plot(newTOD.ravel())
plt.xlim(14000,15000)
plt.show()

In [ ]:
# Veo todo el nuevo TOD
print(np.shape(newTOD.ravel()))
plt.plot(newTOD.ravel()[0:800])
plt.show()

In [ ]:
doit = 1
if doit==1:
	#### Parameters
	ppp = 1./freq_mod
	lowcut = 0.3
	highcut = 10.

	nbins = 250

	reload(dl)
	reload(ft)
	savedir = './'

	ids=0

	for ii in xrange(len(dirs)):
		thedir = dirs[ii]
		print ''
		print '##############################################################'
		print 'Directory {} / {} :'.format(ii, len(dirs)), thedir
		print '##############################################################'
		f0 = glob.glob(savedir+'alltes_unbinned_{}_{}.fits'.format(n,elevations[ii]))
		f1 = glob.glob(savedir+'angles_unbinned_{}_{}.fits'.format(n,elevations[ii]))
		filesalreadydone = ((f0!=[]) & (f1!=[]))
		filesnotdone = bool(not filesalreadydone)
		dothejob = bool(replace or filesnotdone)
		if dothejob:
			if filesalreadydone:
				print 'files already exist on disk but I was asked to replace them so doing the job'
			else: 
				print 'Files do not exist so doing the job'
			allsb = []
			all_az_el_azang = []
			for iasic in [0,1]:
			    print '======== ASIC {} ====================='.format(iasic)
			    AsicNum = iasic+1
			    a = qp()
			    a.read_qubicstudio_dataset(thedir, asic=AsicNum)
			    data=a.azel_etc(TES=None)
			    data['t_src'] += 7200
			    unbinned, binned = dl.general_demodulate(ppp, data, 
			                                            lowcut, highcut,
			                                            nbins=nbins, median=True, method=method, 
			                                            doplot=False, rebin=False, verbose=False)
			    all_az_el_azang.append(np.array([unbinned['az'], unbinned['el'], unbinned['az_ang']]))
			    allsb.append(unbinned['sb'])
			sh0 = allsb[0].shape
			sh1 = allsb[1].shape
			mini = np.min([sh0[1], sh1[1]])
			sb = np.append(allsb[0][:,:mini], allsb[1][:,:mini], axis=0)
			az_el_azang = np.array(all_az_el_azang[0][:,:mini])
			print az_el_azang.shape
			print sb.shape
			FitsArray(sb).save(savedir+'alltes_unbinned_{}_{}.fits'.format(n,elevations[ii]))
			FitsArray(az_el_azang).save(savedir+'angles_unbinned_{}_{}.fits'.format(n,elevations[ii]))
		else:
			if filesalreadydone:
				print 'files already exist on disk and I was asked not to replace them so doing nothing'
			else: 
				print 'This should not happen... There is a bug'

In [ ]:
# Reading real data (already procceced):
def manual_pointing():
    

In [ ]:
print(TOD.shape, data.shape)

In [ ]:
TOD_sim = np.zeros(TOD.shape)
TOD_sim.shape
print(len(TOD_sim[0]), len(data), (len(TOD_sim[0])-len(data))/2 ) 
TOD_sim[0] = np.pad(data, ((len(TOD_sim[0])-len(data))/2,(len(TOD_sim[0])-len(data))/2), 'constant', constant_values=(0., 0.)) 
print(TOD_sim.shape)

In [ ]:
hp.gnomview(data, reso=12)
plt.show()

In [ ]:
print(np.shape(data))
data_ud = hp.ud_grade(data,128)

In [ ]:
print(len(TOD_sim), len(TOD[0]))
plt.plot(TOD_sim[0])
plt.show()

In [ ]:
print(TOD_sim.shape)

maps_recon_real = arec.tod2map(TOD_sim, d, cov=None)


In [ ]:
#maps_recon_real.shape
hp.gnomview(maps_recon_real[:], rot= center, xsize=500,reso=5)
plt.show()